In [2]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup

In [5]:
df_new = pd.read_csv('data/news.csv')
df_new_dir = pd.read_csv('data/news_dir.csv')

In [27]:
df_new.shape
# df_new_dir.shape

(748398, 29)

In [34]:
data_process = pd.merge(df_new[['NewsId','Ticker', 
                       'NewsTitle', 'NewsFullContent', 'PublicDate']],
                 df_new_dir[['NewsId', 'OrganCode']],
                 on='NewsId', how='left')
data_process.head(20)

,NewsId,Ticker,NewsTitle,NewsFullContent,PublicDate,OrganCode
0,4039,NaN,Hết phiên 1: Giảm mạnh nhưng không phá vỡ đáy cũ,<P>Với những lo ngại của các nhà đầu tư trong ...,NaN,NaN
1,4040,NaN,UBCKNN tiếp tục thực hiện nhiều biện pháp giám...,<DIV class=KenhF_Content_News3>\r\n<P></P>\r\n...,NaN,NaN
2,4041,NaN,Ngân hàng làm gì với hàng tỷ USD rút về nước?,"<P>Khủng hoảng tài chính lan rộng, các ngân hà...",NaN,NaN
3,4042,NaN,"Kết thúc ngày giao dịch: VN-Index giảm hơn 1,3%.","Sau phiên giao dịch đầy khởi sắc ngày hôm qua,...",NaN,NaN
4,4043,NaN,VN-Index &quot;phá đáy&quot; trong xu thế giảm...,<DIV class=div_fleft_p10_w250 id=ctl00_Content...,NaN,NaN
5,4044,NaN,Những khúc mắc quanh thuế thu nhập cá nhân với...,<DIV class=KenhF_Content_News3>\r\n<P class=Ms...,NaN,NaN
6,4045,NaN,"Chứng khoán ngày 23/10: Lập đáy mới, chờ tín hiệu",<P>Nhiều nhà đầu tư chờ khối ngoại mua ròng tr...,NaN,NaN
7,4046,NaN,Có hay không việc nhà đầu tư ngoại rút vốn?,<DIV class=KenhF_Content_News3>\r\n<P><SPAN st...,NaN,NaN
8,4047,NaN,Kết thúc ngày 24/10: VN-Index đã xuống gần mốc...,<P>Sau khi thủng đáy cũ 366 trong phiên giao d...,NaN,NaN
9,4048,NaN,Phố Wall ngóng chờ quyết định của FED,"<FONT face=""arial, helvetica, sans-serif""><FON...",NaN,NaN


In [41]:
data_process.columns
data_process.shape
data_process['Ticker'].fillna(data_process['OrganCode'], inplace=True)
data_process.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 751699 entries, 0 to 751698
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   NewsId           751699 non-null  int64 
 1   Ticker           586139 non-null  object
 2   NewsTitle        751699 non-null  object
 3   NewsFullContent  751530 non-null  object
 4   PublicDate       751652 non-null  object
 5   OrganCode        366593 non-null  object
dtypes: int64(1), object(5)
memory usage: 60.1+ MB


In [74]:
data_process.head(10)

,NewsId,Ticker,NewsTitle,NewsFullContent,PublicDate,OrganCode
0,4039,NaN,Hết phiên 1: Giảm mạnh nhưng không phá vỡ đáy cũ,Với những lo ngại của các nhà đầu tư trong nướ...,NaN,NaN
1,4040,NaN,UBCKNN tiếp tục thực hiện nhiều biện pháp giám...,Bên lề cuộc hội thảo “Chuyển đổi tài khoản nhà...,NaN,NaN
2,4041,NaN,Ngân hàng làm gì với hàng tỷ USD rút về nước?,"Khủng hoảng tài chính lan rộng, các ngân hàng ...",NaN,NaN
3,4042,NaN,"Kết thúc ngày giao dịch: VN-Index giảm hơn 1,3%.","Sau phiên giao dịch đầy khởi sắc ngày hôm qua,...",NaN,NaN
4,4043,NaN,VN-Index &quot;phá đáy&quot; trong xu thế giảm...,(CafeF) - Thị trường chứng khoán thế giới đã t...,NaN,NaN
5,4044,NaN,Những khúc mắc quanh thuế thu nhập cá nhân với...,Thị trường chứng khoán “xôn xao” với việc thuế...,NaN,NaN
6,4045,NaN,"Chứng khoán ngày 23/10: Lập đáy mới, chờ tín hiệu",Nhiều nhà đầu tư chờ khối ngoại mua ròng trở l...,NaN,NaN
7,4046,NaN,Có hay không việc nhà đầu tư ngoại rút vốn?,"Thưa ông, xin ông chi biết lý do của việc ngoạ...",NaN,NaN
8,4047,NaN,Kết thúc ngày 24/10: VN-Index đã xuống gần mốc...,Sau khi thủng đáy cũ 366 trong phiên giao dịch...,NaN,NaN
9,4048,NaN,Phố Wall ngóng chờ quyết định của FED,Chứng khoán Mỹ tiếp tục giảm mạnhGiá dầu kỳ hạ...,NaN,NaN


In [116]:
data_process = data_process.dropna(subset=['NewsFullContent']).reset_index()

In [ ]:

data_process['NewsFullContent'][1000]

In [119]:
def cleanhtml(raw_html):
    for i in range(len(raw_html)):
        cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|\n|\t|\r|\xa0|&nbsp;')
        raw_html[i] = re.sub(cleanr,'', str(raw_html[i]))
        raw_html[i] = BeautifulSoup(raw_html[i], "lxml").text
    return raw_html

In [120]:
data_array_clean = cleanhtml(data_process['NewsFullContent'])
data_process['NewsFullContent'] = data_array_clean

KeyboardInterrupt: 

In [ ]:
data_process.shape

In [189]:
data_process.to_csv('data/data_preprocess.csv', encoding='utf-8')